In [3]:
import os
os.chdir('/Volumes/Transcend/00unzip')
import json
import ijson

import pandas as pd

In [16]:


f = open('/Volumes/Transcend/00unzip/gab_posts_jan_2018.json')
objects = ijson.items(f,'actuser.username')

chunk=[]
username=[]
body=[]
ts=[]

i = 0
for a in f:
    line = json.loads(a)
    username.append(line['actuser']['username'])
    body.append(line['post']['body'])
    ts.append(line['published_at'])
    i = i+1
    if i > 100000:
        break
                      
df = pd.DataFrame(
    {'username': username,
     'body': body,
     'ts': ts
    })
df.head()

,username,body,ts
0,Hogantwonk,What anyone thinks of guns doesn’t matter. It ...,2017-11-06T09:09:28+00:00
1,TwistedMemes,💔,2017-11-06T09:01:07+00:00
2,RealTrumpTweets,...Americans do what we do best: we pull toget...,2017-11-06T07:27:45+00:00
3,Fash_McQueen,It's becoming impossible to parody the Alt-Lit...,2017-11-06T09:09:16+00:00
4,RaviCrux,The media is complicit.\nThey embolden these l...,2017-11-06T07:15:04+00:00


In [17]:
import sys
df.to_csv('gab_snippet.csv')
#100K record is ~18mb